Import library yang dibutuhkan

In [3]:
import pandas as pd
import re
import numpy as np

Import dan mengecek data

In [4]:
df = pd.read_csv("data//yogya_hp.csv")

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2020 entries, 0 to 2019
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   price             2020 non-null   object 
 1   nav-link          2020 non-null   object 
 2   description       2020 non-null   object 
 3   listing-location  2020 non-null   object 
 4   bed               2001 non-null   float64
 5   bath              1999 non-null   float64
 6   carport           1713 non-null   float64
 7   surface_area      2019 non-null   object 
 8   building_area     2019 non-null   object 
dtypes: float64(3), object(6)
memory usage: 142.2+ KB


In [6]:
df.head()

,price,nav-link,description,listing-location,bed,bath,carport,surface_area,building_area
0,"Rp 1,79 Miliar",https://www.rumah123.com/properti/sleman/hos17...,Rumah 2 Lantai Baru di jalan Palagan Sleman Y...,"Ngaglik, Sleman",3.0,3.0,2.0,120 m²,110 m²
1,Rp 170 Juta,https://www.rumah123.com/properti/sleman/hos17...,RUMAH BARU DEKAT AL AZHAR DAN UGM,"Jombor, Sleman",3.0,2.0,1.0,102 m²,126 m²
2,Rp 695 Juta,https://www.rumah123.com/properti/sleman/hos17...,RUMAH ASRI DAN SEJUK DI BERBAH SLEMAN DEKAT PA...,"Berbah, Sleman",2.0,2.0,1.0,100 m²,100 m²
3,Rp 560 Juta,https://www.rumah123.com/properti/sleman/hos17...,Rumah Murah 5 Menit Dari Candi Prambanan Tersi...,"Prambanan, Sleman",3.0,1.0,1.0,109 m²,67 m²
4,Rp 200 Juta,https://www.rumah123.com/properti/sleman/hos17...,Rumah Murah Cicilan 1jt Di Moyudan Sleman,"Moyudan, Sleman",2.0,1.0,1.0,60 m²,30 m²


Dari sini parameter yang ingin kita gunakan adalah harga sebagai parameter target (Y) dan lokasi, jumlah kamar tidur(bed), kamar mandi (bath), jumlah parkir mobil (carport), luas tanah (surface_area), dan luas bangunan (building_area)

Cleaning Data

In [7]:
#Function untuk cleaning data
def clean_property_data(df):
    df_cleaned = df.copy()
    # Cleaning data price
    def clean_price(price_str):
        if pd.isna(price_str):
            return None
            
        # Menghapus Rp dan spasi
        price_str = str(price_str).replace('Rp', '').strip()
        
        # mengganti koma menjadi titik agar dimengerti program
        price_str = price_str.replace(',', '.')
        
        # Handle Miliar 
        if 'Miliar' in price_str:
            number = float(re.findall(r'[\d.]+', price_str)[0])
            return int(number * 1_000_000_000)
        
        # Handle Juta   
        elif 'Juta' in price_str:
            number = float(re.findall(r'[\d.]+', price_str)[0])
            return int(number * 1_000_000)
        
        # Handle plain numbers
        else:
            try:
                return int(float(price_str))
            except ValueError:
                return None
    
        

    # menghapus m^2
    def clean_area(area_str):
        if pd.isna(area_str):
            return None
        area_str = str(area_str).replace('m²', '').strip()
        try:
            return int(float(area_str))
        except ValueError:
            return None
    #Kita hanya mengambil data di sleman
    df_cleaned = df[df['listing-location'].str.contains(r', Sleman', na=False)]
    #apply function
    df_cleaned['price'] = df_cleaned['price'].apply(clean_price)
    df_cleaned['surface_area'] = df_cleaned['surface_area'].apply(clean_area)
    df_cleaned['building_area'] = df_cleaned['building_area'].apply(clean_area)
    df_cleaned = df_cleaned.dropna()
    df_cleaned =df_cleaned.drop_duplicates()
    return df_cleaned

In [8]:
df.drop(["nav-link","description"], axis = 1, inplace=True) #menghapus tabel yang tidak dibutuhkan

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2020 entries, 0 to 2019
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   price             2020 non-null   object 
 1   listing-location  2020 non-null   object 
 2   bed               2001 non-null   float64
 3   bath              1999 non-null   float64
 4   carport           1713 non-null   float64
 5   surface_area      2019 non-null   object 
 6   building_area     2019 non-null   object 
dtypes: float64(3), object(4)
memory usage: 110.6+ KB


In [10]:
df = clean_property_data(df) #cleaning data

C:\Users\adria\AppData\Local\Temp\ipykernel_1948\766707705.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['price'] = df_cleaned['price'].apply(clean_price)
C:\Users\adria\AppData\Local\Temp\ipykernel_1948\766707705.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['surface_area'] = df_cleaned['surface_area'].apply(clean_area)
C:\Users\adria\AppData\Local\Temp\ipykernel_1948\766707705.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

In [11]:
df.head()

,price,listing-location,bed,bath,carport,surface_area,building_area
0,1790000000,"Ngaglik, Sleman",3.0,3.0,2.0,120.0,110.0
1,170000000,"Jombor, Sleman",3.0,2.0,1.0,102.0,126.0
2,695000000,"Berbah, Sleman",2.0,2.0,1.0,100.0,100.0
3,560000000,"Prambanan, Sleman",3.0,1.0,1.0,109.0,67.0
4,200000000,"Moyudan, Sleman",2.0,1.0,1.0,60.0,30.0


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 859 entries, 0 to 2017
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   price             859 non-null    int64  
 1   listing-location  859 non-null    object 
 2   bed               859 non-null    float64
 3   bath              859 non-null    float64
 4   carport           859 non-null    float64
 5   surface_area      859 non-null    float64
 6   building_area     859 non-null    float64
dtypes: float64(5), int64(1), object(1)
memory usage: 53.7+ KB


In [13]:
df['listing-location'].value_counts()

listing-location
Ngaglik, Sleman              216
Depok, Sleman                127
Mlati, Sleman                 82
Kalasan, Sleman               72
Gamping, Sleman               66
Sleman, Sleman                62
Ngemplak, Sleman              61
Godean, Sleman                29
Condong Catur, Sleman         26
Purwomartani   , Sleman       25
Berbah, Sleman                19
Kaliurang, Sleman             16
Prambanan, Sleman             15
Sayegan, Sleman               10
Caturtunggal, Sleman           8
Pakem, Sleman                  8
Cebongan, Sleman               5
Moyudan, Sleman                3
Tempel, Sleman                 3
Turi, Sleman                   3
Jombor, Sleman                 2
Sidoarum         , Sleman      1
Name: count, dtype: int64

Karena semua data sudah hanya di sleman, kita bisa menghapus Sleman

In [14]:
df['listing-location'] = df['listing-location'].str.replace('Sleman', '', regex=False) \
                                               .str.replace(',', '', regex=False) \
                                               .str.replace(' ', '', regex=False)


In [15]:
df['listing-location'].value_counts()

listing-location
Ngaglik         216
Depok           127
Mlati            82
Kalasan          72
Gamping          66
                 62
Ngemplak         61
Godean           29
CondongCatur     26
Purwomartani     25
Berbah           19
Kaliurang        16
Prambanan        15
Sayegan          10
Caturtunggal      8
Pakem             8
Cebongan          5
Moyudan           3
Tempel            3
Turi              3
Jombor            2
Sidoarum          1
Name: count, dtype: int64

Ini menyebabkan Sleman(kecamatan), Sleman(Kabupaten) meenjadi no string, mari kita ganti kembali

In [16]:
df['listing-location'] = df['listing-location'].replace('', 'Sleman')

In [17]:
df['listing-location'].value_counts()

listing-location
Ngaglik         216
Depok           127
Mlati            82
Kalasan          72
Gamping          66
Sleman           62
Ngemplak         61
Godean           29
CondongCatur     26
Purwomartani     25
Berbah           19
Kaliurang        16
Prambanan        15
Sayegan          10
Caturtunggal      8
Pakem             8
Cebongan          5
Moyudan           3
Tempel            3
Turi              3
Jombor            2
Sidoarum          1
Name: count, dtype: int64

In [18]:
df.drop(columns='listing-location').agg(['min', 'max', 'mean'])

,price,bed,bath,carport,surface_area,building_area
min,1.550000e+08,1.00000,1.000000,1.000000,36.000000,4.000000e+00
max,2.500000e+10,40.00000,42.000000,15.000000,4222.000000,2.408150e+09
mean,1.805024e+09,4.19092,3.323632,1.621653,187.905704,2.803590e+06


Nampaknya ada outlier mari kita coba remove

In [19]:
cols_to_check = ['bed','bath','price','surface_area', 'building_area']
def remove_outliers_iqr(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR

        # Filter the DataFrame
        df = df[(df[col] >= lower) & (df[col] <= upper)]
    return df

df = remove_outliers_iqr(df, cols_to_check)

In [20]:
df.drop(columns='listing-location').agg(['min', 'max', 'mean'])

,price,bed,bath,carport,surface_area,building_area
min,1.550000e+08,2.000000,1.000000,1.00000,36.000000,4.000000
max,3.370000e+09,5.000000,6.000000,4.00000,209.000000,235.000000
mean,1.147210e+09,2.967593,2.100309,1.28858,119.564815,96.492284


In [21]:
df.head()

,price,listing-location,bed,bath,carport,surface_area,building_area
0,1790000000,Ngaglik,3.0,3.0,2.0,120.0,110.0
1,170000000,Jombor,3.0,2.0,1.0,102.0,126.0
2,695000000,Berbah,2.0,2.0,1.0,100.0,100.0
3,560000000,Prambanan,3.0,1.0,1.0,109.0,67.0
4,200000000,Moyudan,2.0,1.0,1.0,60.0,30.0


Data sudah bersih sesuai yang kita inginkan, lalu akan kita save

In [22]:
df.to_csv('data/cleaned_yogya_hp.csv', index=False)
